In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

This dataset contains all bitcoin exchanges for the time period of Jan 2012 to September 2020, with minute to minute updates of OHLC (Open, High, Low, Close), Volume in BTC and indicated currency, and weighted bitcoin price. Timestamps are in Unix time. Timestamps without any trades or activity have their data fields filled with Nan's.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime

In [ ]:
df = pd.read_csv('../input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2020-09-14.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

Dropping empty rows

In [ ]:
df.dropna(axis=0,how='any',inplace=True)
df.head()

In [ ]:
df.shape

Converting 'Timestamp' to date-time format and setting it as the index of our data

In [ ]:
df['Timestamp'] = [datetime.fromtimestamp(x) for x in df['Timestamp']]
df = df.set_index(['Timestamp'])

df.head()

In [ ]:
df.isnull().values.any()

Now I am selecting only those rows from where the 'Weighted_Price' is greater than 2500 because before the price 2500 the data was almost stationary and it would effect the model performance.

In [ ]:
df = df.loc[df['Weighted_Price'] >2500]

In [ ]:
data = df['Weighted_Price']
data.head()

In [ ]:
plt.plot(data)
plt.title('Visualising Weighted Prices over time',fontsize=18)
plt.xlabel('Time',fontsize=15)
plt.ylabel('Price value',fontsize=15)
plt.xticks(rotation=45)
plt.show()

Normalising values using Standard Scaler

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

data = sc.fit_transform(np.array(data).reshape(-1,1))

data[:5]

Splitting our data into training and testing

In [ ]:
training_size = int(len(data)*0.65)

training_data = data[0:training_size,:]
testing_data = data[training_size:,:]

print(training_data.shape)
print(testing_data.shape)

Arranging the data in such a way that the value of current day depends on the previous value.

Here two lists are created for X and Y.The first value of the dataset is added in X and the second value is added in Y because our current prediction depends on the  previous value i.e the timestamp here is 1.Now as the loop continues the second values is then appended in X and third value in Y and so on.Once we reach at the end of the loop the last value of X will be the second last value of our dataset and last value of Y will be the last value of our dataset.

In [ ]:
def create_dataset(data,timestamp=1):
    X=[]
    Y=[]
    
    for i in range(len(data)-timestamp-1):
        a = data[i:(i+timestamp)]
        X.append(a)
        
        Y.append(data[i+timestamp])
        
    return np.array(X),np.array(Y)
        
        

In [ ]:
timestamp=1
X_train,Y_train = create_dataset(training_data,timestamp)
x_test,y_test = create_dataset(testing_data,timestamp)

print(X_train.shape)
print(Y_train.shape)
print(x_test.shape)
print(y_test.shape)


In [ ]:
print(X_train[:6])

print(Y_train[:6])

Creating LSTM model for prediction

In [ ]:
from tensorflow.keras import models
from tensorflow.keras.layers import LSTM,Dense,Dropout


In [ ]:
model = models.Sequential()

model.add(LSTM(50,return_sequences = True,input_shape = (5,1),activation='relu'))
model.add(LSTM(25,return_sequences = True))
model.add(LSTM(16,return_sequences = True))
model.add(LSTM(10,return_sequences = False))

model.add(Dense(1))

model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()

Training model

In [ ]:
hist = model.fit(X_train,Y_train,epochs=10,batch_size=64,validation_split=0.35)

In [ ]:
hist.history

Visualising training and validation loss.

In [ ]:
training_loss = hist.history['loss']
val_loss = hist.history['val_loss']

plt.plot(training_loss,label='training_loss')
plt.plot(val_loss,label='val_loss')
plt.legend()
plt.title('Visualising loss',fontsize=18)
plt.xlabel('Epochs',fontsize=15)
plt.ylabel('Loss',fontsize=15)
plt.show()

Making predictions on test data

In [ ]:
predictions = model.predict(x_test)

Converting the predicted and original test values into their original form

In [ ]:
y_test = sc.inverse_transform(y_test)
y_pred = sc.inverse_transform(predictions)

In [ ]:
plt.plot(y_test,label='Original Values')
plt.plot(y_pred,label='Predicted Values')
plt.legend()
plt.title('Original vs Predicted values curve',fontsize=18)
plt.xlabel('Number of test points',fontsize=15)
plt.ylabel('Weighted_Price',fontsize=15)
plt.plot()

In [ ]:
from sklearn.metrics import r2_score

score = r2_score(y_test,y_pred)
score

Here we can see both the curves have overlapped and the r2_score is almost 1 which tells us that the model is able to generate good results.

Please give an upvote if you found this notebook useful and any other type of suggestions are most welcome.